In [109]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [130]:
Experiment.objects.get(name='sats_games').delete()

(4,
 {'auctions.BestResponse': 1,
  'auctions.EquilibriumSolverRunCheckpoint': 1,
  'auctions.EquilibriumSolverRun': 1,
  'auctions.Experiment': 1})

In [122]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]
sats_games = [os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/sats*')]
sats_games

['sats_50', 'sats_5', 'sats_10', 'sats_100', 'sats_20']

In [123]:
# SATS Games
CONFIG = 'april12'
N_SEEDS = 1
overrides = f'--overwrite_db True --br_portfolio_path {CONFIG} --eval_every 1_000_000 --eval_zero true --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
for GAME_NAME in sats_games:
    dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

Submitted batch job 29559
Submitted batch job 29560
Submitted batch job 29561
Submitted batch job 29562
Submitted batch job 29563
Submitted batch job 29564
Submitted batch job 29565
Submitted batch job 29566
Submitted batch job 29567
Submitted batch job 29568
Submitted batch job 29569
Submitted batch job 29570
Submitted batch job 29571
Submitted batch job 29572
Submitted batch job 29573


In [ ]:
# game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]
# albert_games = [os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/rw*')]
# albert_games

['rw_game_3_settle',
 'rw_game_2_war_95',
 'rw_game_3_settle_hide',
 'rw_game_2_war_hide',
 'rw_game_2_war',
 'rw_game_2_war_97',
 'rw_game_2_settle',
 'rw_game_2_settle_no_undersell',
 'rw_game_2_war_99',
 'rw_game_2_settle_hide',
 'rw_game_3_settle_marginal']

In [121]:
# # Albert's games
# CONFIG = 'april12'
# N_SEEDS = 1
# overrides = f'--overwrite_db True --br_portfolio_path {CONFIG} --eval_every 1_000_000 --eval_zero false --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000" --random_ic true'
# for GAME_NAME in albert_games:
#     dispatch.dispatch_experiments(CONFIG, base_job_name='albert_games4', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

Submitted batch job 29214
Submitted batch job 29215
Submitted batch job 29216


In [56]:
CONFIG = 'flatmlp'
GAME_NAME = 'large_game_2'
N_SEEDS = 1
overrides = f'--overwrite_db True --br_portfolio_path flatmlp --eval_every 1_000_000 --eval_zero true --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
dispatch.dispatch_experiments(CONFIG, base_job_name='april11_flattest_large_2', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

Submitted batch job 24595
Submitted batch job 24596
Submitted batch job 24597
Submitted batch job 24598
Submitted batch job 24599
Submitted batch job 24600


In [67]:
# CONFIG = 'flatmlp'
# GAME_NAME = 'very_large_game_2'
# N_SEEDS = 1
# overrides = f'--overwrite_db True --br_portfolio_path flatmlp --eval_every 1_000_000 --eval_zero true --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'
# dispatch.dispatch_experiments(CONFIG, base_job_name='april11_flattest_very_large_2', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

In [ ]:
CONFIG = 'mar17'
GAME_NAME = 'large_game_2'
N_SEEDS = 5
overrides = f'--br_portfolio_path mar17 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'

# Base
dispatch.dispatch_experiments(CONFIG, base_job_name='mar24', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

# Hidden demands
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_hide', overrides = overrides, n_seeds=N_SEEDS)

# Reserve prices
# TODO: Holding off for now b/c rounds issue
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_low_reserve', overrides = overrides, n_seeds=N_SEEDS)
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_same_reserve', overrides = overrides, n_seeds=N_SEEDS)

# Activity
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_activity', overrides = overrides, n_seeds=N_SEEDS)
# Undersell
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_undersell', overrides = overrides, n_seeds=N_SEEDS)

# Initial pricing experiment, on the base game
dispatch.dispatch_experiments(CONFIG, base_job_name='mar24', game_name=f'{GAME_NAME}_pricing05', overrides = overrides, n_seeds=N_SEEDS)

In [ ]:
CONFIG = 'mar17'
GAME_NAME = 'large_game_3b'
N_SEEDS = 5
overrides = f'--br_portfolio_path mar17 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000"'

# Base
dispatch.dispatch_experiments(CONFIG, base_job_name='mar22_4', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)

# Hidden demands
dispatch.dispatch_experiments(CONFIG, base_job_name='mar22_4', game_name=f'{GAME_NAME}_hide', overrides = overrides, n_seeds=N_SEEDS)

# Reserve prices
# TODO: Holding off for now b/c rounds issue
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_low_reserve', overrides = overrides, n_seeds=N_SEEDS)
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_same_reserve', overrides = overrides, n_seeds=N_SEEDS)

# Activity
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_activity', overrides = overrides, n_seeds=N_SEEDS)
# Undersell
# dispatch.dispatch_experiments(CONFIG, base_job_name='mar19', game_name=f'{GAME_NAME}_no_undersell', overrides = overrides, n_seeds=N_SEEDS)

# Initial pricing experiment, on the base game
dispatch.dispatch_experiments(CONFIG, base_job_name='mar22_4', game_name=f'{GAME_NAME}_pricing05', overrides = overrides, n_seeds=N_SEEDS)

In [ ]:
pks = list(map(str, EquilibriumSolverRunCheckpoint.objects.filter(equilibrium_solver_run__experiment__name='infocompare4', equilibrium_solver_run__game__name='large_game_2.json', t=1e7, equilibrium_solver_run__name__contains='mlp').values_list('pk', flat=True)))
CONFIG = 'feb22'
GAME_NAME = 'large_game_2'
N_SEEDS = 1
overrides = f'--br_portfolio_path feb22 --eval_every 1_000_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000" --rnr_player 0 --rnr_checkpoints {" ".join(pks)}'

dispatch.dispatch_experiments(CONFIG, base_job_name='rnrtest', game_name=GAME_NAME, overrides = overrides, n_seeds=N_SEEDS)
# # dispatch.dispatch_experiments('feb22', base_job_name='infocompare3', game_name='large_game_2_hide', overrides = overrides, n_seeds=5)

# dispatch.dispatch_experiments('mar14', base_job_name='mar15_brier_lr', game_name='large_game_2', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('mar10', base_job_name='switch_and_brier3', game_name='large_game_2_switch10', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('feb22', base_job_name='switch_and_brier3_old_config', game_name='large_game_2_switch10', overrides = overrides, n_seeds=1)

# dispatch.dispatch_experiments('feb22', base_job_name='mar14_very_large_game_2', game_name='very_large_game_2', overrides = overrides, n_seeds=1)
# dispatch.dispatch_experiments('feb22', base_job_name='mar14_large_game_3', game_name='large_game_3', overrides = overrides, n_seeds=5)
# dispatch.dispatch_experiments('feb22', base_job_name='mar14_large_game_3', game_name='large_game_3_hide', overrides = overrides, n_seeds=5)
# dispatch.dispatch_experiments('feb22', base_job_name='tiny_game_Test', game_name='tiny', overrides = overrides, n_seeds=1)

In [ ]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "rnrtest" | awk '{print $1}' | xargs scancel

In [128]:
!squeue --format="%.18i %.80j %.2t .%+.10M %.6D %R"

squeue: error: Invalid job format specification: +
             JOBID                                                                             NAME ST .%+.10M  NODES NODELIST(REASON)
             29565                                                sats_10-april12mlp-100_sats_games PD .%+.10M      1 (Resources)
             29566                                               sats_10-april12lstm-100_sats_games PD .%+.10M      1 (Priority)
             29567                                   sats_10-april12flat_handcrafted-100_sats_games PD .%+.10M      1 (Priority)
             29568                                               sats_100-april12mlp-100_sats_games PD .%+.10M      1 (Priority)
             29569                                              sats_100-april12lstm-100_sats_games PD .%+.10M      1 (Priority)
             29570                                  sats_100-april12flat_handcrafted-100_sats_games PD .%+.10M      1 (Priority)
             29571                     

In [ ]:
!squeue | wc -l

In [129]:
!scancel -u ubuntu

In [98]:
# !scancel  25809

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
from open_spiel.python.examples.ubc_plotting_utils import parse_run
from auctions.webutils import *

def find_best_checkpoint(run, max_t=None):
    ev_df = parse_run(run, max_t)
    best_t = ev_df.groupby('t')['ApproxNashConv'].first().idxmin()
    nash_conv_by_t = ev_df.groupby('t')['ApproxNashConv'].first()
    best_checkpoint = get_checkpoint(run, t=best_t)
    return nash_conv_by_t, best_checkpoint, nash_conv_by_t.min()


run = EquilibriumSolverRun.objects.last()
find_best_checkpoint(run)

In [ ]:
Experiment.objects.get(name='large_game_2_no_activity.json').delete()

In [ ]:
Game.objects.get(name='large_game_2_pricing_extreme.json').delete()

In [ ]:
!python /apps/open_spiel/web/auctions/manage.py nfsp --seed 109 --filename rounds.json --network_config_file mar17/lstm --experiment_name lstmtest --job_name "lstm_for_greg" --dispatch_br false --br_portfolio_path feb22 --eval_every 10_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000" --report_freq 10_000 --overwrite_db true

In [ ]:
list(Experiment.objects.all().values_list('name', flat=True))

In [ ]:
r = EquilibriumSolverRun.objects.get(name='large_game_3b_hide-mar17lstm-100')

In [ ]:
from auctions.webutils import *
from open_spiel.python.examples.ubc_plotting_utils import *
parse_run(r, conservative=False)

In [ ]:
BREvaluation.objects.filter(best_response__checkpoint__equilibrium_solver_run=r)

In [ ]:
list(BestResponse.objects.filter(checkpoint__equilibrium_solver_run=r).values('name'))

In [ ]:
BestResponse.objects.filter(checkpoint=pk)[0].brevaluation.__dict__

In [ ]:
from open_spiel.python.examples.ubc_plotting_utils import *
parse_run(r, conservative=False)

In [ ]:
BREvaluation.objects.filter(best_response__checkpoint__equilibrium_solver_run=r).values('best_response__name')

In [ ]:
BestResponse.objects.filter(checkpoint__equilibrium_solver_run=r).values('name')

In [ ]:
experiments = ['mar24', 'mar22_4']
all_scripts = []
for e in experiments:
    scripts = glob.glob(f'/shared/outputs/{e}/*/evaluations/*.sh')
    scripts = [s for s in scripts if 'straightforward' not in s]
    scripts = [s for s in scripts if s[-5:] == "_2.sh"]
    all_scripts += scripts

In [ ]:
for script in all_scripts:
    parent = Path(script).parent
    !cd {parent} && sbatch {script}

In [ ]:
x = Evaluation.objects.last()

In [ ]:
x.samples['allocations']

In [17]:
Evaluation.objects.last()

<Evaluation: Evaluation for large_game_2-april1lstm-100 (april7_test_3) Iteration 1>

In [8]:
import torch.nn.functional as F
import torch
x = torch.tensor([1,2,3])

In [15]:
F.pad(x, (0, 2))

tensor([1, 2, 3, 0, 0])

In [ ]:
torch.nn.functional.pad(input, pad, mode='constant', value=0.0)

In [89]:
x =np.array([[1,2,3],[4,5,6]])

In [90]:
x.sum(axis=0)

array([5, 7, 9])

In [91]:
x.sum(axis=1)

array([ 6, 15])

In [92]:
np.array([5,10]).sum(axis=1)

AxisError: axis 1 is out of bounds for array of dimension 1